In [33]:
pip install keras tensorflow opencv-python

Note: you may need to restart the kernel to use updated packages.


In [34]:
import os
from keras.preprocessing import image
import cv2


categories = ['with_mask', 'without_mask']

data = []

for category in categories:
    path = os.path.join('train',category)
    # print(path,os.listdir(path))

    label = categories.index(category)

    for file in os.listdir(path):
        image_path = os.path.join(path,file)
        img = cv2.imread(image_path)
        img = cv2.resize(img,(224,224))

        data.append([img,label])







In [40]:
len(data)

1315

In [39]:

import random

random.shuffle(data)

x=[]
y=[]

for features,label in data:
    x.append(features)
    y.append(label)



2630

In [37]:
import numpy as np

x = np.array(x)
y = np.array(y)

x.shape

(2630,)